In [ ]:
!pip install kagglehub


#kaggle.json từ kaggle

In [ ]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"tranlong0814","key":"a12e7bc7b2f177e577dc431a7226a6c1"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:

import os
import kagglehub

path = kagglehub.dataset_download("akaiinu/crema-d")
print("Path downloaded:", path)

# Kiểm tra xem trong path có gì
print(os.listdir(path))


Using Colab cache for faster access to the 'crema-d' dataset.
Path downloaded: /kaggle/input/crema-d
['cremad']


#tải bộ dữ liệu từ kaggle


In [ ]:
!kaggle datasets download -d akaiinu/crema-d
!unzip -q crema-d.zip -d ./crema-d


Dataset URL: https://www.kaggle.com/datasets/akaiinu/crema-d
License(s): MIT
User cancelled operation
replace ./crema-d/cremad/AudioWAV/1001_DFA_ANG_XX.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
root_dir = "./crema-d/cremad/Image-01-FPS"

In [ ]:
import os, glob, pandas as pd

rows = []
for folder in os.listdir(root_dir):
    folder_path = os.path.join(root_dir, folder)
    if not os.path.isdir(folder_path):
        continue

    # tách nhãn cảm xúc
    parts = folder.split("_")
    if len(parts) < 3:
        continue
    emotion = parts[2]  # ANG, DIS, FEA, HAP, NEU, SAD

    for img_path in glob.glob(os.path.join(folder_path, "*.jpg")):
        rows.append([img_path, emotion])

df = pd.DataFrame(rows, columns=["image_path", "emotion"])
df = df[df['emotion'].isin(['ANG', 'DIS', 'FEA', 'HAP', 'NEU', 'SAD'])]  # 6 nhãn chính
print("Số lượng ảnh:", len(df))
print("Các nhãn:", sorted(df["emotion"].unique()))
df.head()


Số lượng ảnh: 23299
Các nhãn: ['ANG', 'DIS', 'FEA', 'HAP', 'NEU', 'SAD']


,image_path,emotion
0,./crema-d/cremad/Image-01-FPS/1019_TAI_HAP_XX/...,HAP
1,./crema-d/cremad/Image-01-FPS/1019_TAI_HAP_XX/...,HAP
2,./crema-d/cremad/Image-01-FPS/1019_TAI_HAP_XX/...,HAP
3,./crema-d/cremad/Image-01-FPS/1028_IOM_NEU_XX/...,NEU
4,./crema-d/cremad/Image-01-FPS/1028_IOM_NEU_XX/...,NEU


In [ ]:
# Import libraries
import os,cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from pylab import rcParams
rcParams['figure.figsize'] = 40, 40
import random

In [ ]:

emotions = sorted(df["emotion"].unique())
emo2idx = {emo: i for i, emo in enumerate(emotions)}
df["label"] = df["emotion"].map(emo2idx)
print(emo2idx)


{'ANG': 0, 'DIS': 1, 'FEA': 2, 'HAP': 3, 'NEU': 4, 'SAD': 5}


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms

# Map nhãn
emotion_to_label = {'ANG': 0, 'DIS': 1, 'FEA': 2, 'HAP': 3, 'NEU': 4, 'SAD': 5}

# Transform ảnh
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # chuẩn ImageNet
                         std=[0.229, 0.224, 0.225])
])

# Dataset
class CREMADataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = row['image_path']
        label = emotion_to_label[row['emotion']]

        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)

        return image, label

# Split train/test
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['emotion'], random_state=42)

train_dataset = CREMADataset(train_df, transform=transform)
test_dataset = CREMADataset(test_df, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=2)

print(f"Train samples: {len(train_dataset)}, Test samples: {len(test_dataset)}")


Train samples: 18639, Test samples: 4660


In [ ]:
# Cài đặt facenet-pytorch
!pip install facenet-pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 116.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 126.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 114.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 785.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

#Trích xuất đặc trưng bằng Resnet50


In [ ]:
import os
import torch
from facenet_pytorch import MTCNN
from torchvision import models, transforms
from torch import nn
from PIL import Image
import pickle
from tqdm import tqdm

from torchvision.transforms import functional as TF

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Face detector (crop face)
mtcnn = MTCNN(image_size=224, margin=0, device=device)

# Load ResNet50 pretrained ImageNet
resnet = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
resnet = resnet.to(device)
resnet.eval()

# Bỏ layer FC -> lấy feature 2048-dim
feature_extractor = nn.Sequential(*list(resnet.children())[:-1]).to(device)

# Chuẩn hóa theo ImageNet
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

root_dir = "./crema-d/cremad/Image-01-FPS"
all_features = {}

for video_folder in tqdm(sorted(os.listdir(root_dir))):
    video_path = os.path.join(root_dir, video_folder)
    if not os.path.isdir(video_path):
        continue

    frames = sorted([f for f in os.listdir(video_path)
                     if f.lower().endswith(('.jpg', '.png'))])

    selected_frames = frames[::10]

    batch_imgs = [Image.open(os.path.join(video_path, f)).convert('RGB')
                  for f in selected_frames]

    # Face detection (crop)
    faces = mtcnn(batch_imgs)

    for img_name, face in zip(selected_frames, faces):
        img_path = os.path.join(video_path, img_name)

        if face is None:
            print(f"Không detect được mặt: {img_path}")
            continue

        # Đưa tensor mặt lên GPU (nếu có)
        face = face.to(device)

        # Convert tensor → PIL
        face_pil = TF.to_pil_image(face.cpu())

        # Transform chuẩn ImageNet + đưa lên GPU
        face_tensor = transform(face_pil).unsqueeze(0).to(device)

        with torch.no_grad():
            feat = feature_extractor(face_tensor)   # <-- giờ không còn lỗi
            feat = feat.squeeze().cpu().numpy()

        all_features[img_path] = feat


# Lưu feature
with open("cremad_resnet50_features.pkl", "wb") as f:
    pickle.dump(all_features, f)

print("Đã trích xuất xong:", len(all_features), "ảnh")


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 160MB/s]
 51%|█████     | 3768/7442 [02:48<02:52, 21.33it/s]

Không detect được mặt: ./crema-d/cremad/Image-01-FPS/1047_IEO_FEA_LO/00000.jpg


 51%|█████     | 3777/7442 [02:49<02:56, 20.72it/s]

Không detect được mặt: ./crema-d/cremad/Image-01-FPS/1047_IEO_SAD_LO/00000.jpg


 83%|████████▎ | 6192/7442 [04:36<00:49, 25.24it/s]

Không detect được mặt: ./crema-d/cremad/Image-01-FPS/1076_MTI_SAD_XX/00000.jpg


100%|██████████| 7442/7442 [05:31<00:00, 22.48it/s]


Đã trích xuất xong: 7445 ảnh


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import shutil

src = "/content/cremad_resnet50_features.pkl"
dst = "/content/drive/MyDrive"

shutil.copy(src, dst)
print("Đã lưu vào:", dst)


Đã lưu vào: /content/drive/MyDrive


In [ ]:
def extract_features(dataloader):
    features = []
    labels = []

    with torch.no_grad():  # tắt gradient để tiết kiệm RAM/GPU
        for imgs, labs in dataloader:
            imgs = imgs.to(device)
            outputs = resnet(imgs)                  # (B, 512, 1, 1) hoặc (B, 2048, 1, 1)
            outputs = outputs.view(outputs.size(0), -1)  # flatten -> (B, 512)

            features.append(outputs.cpu().numpy())  # đưa về CPU và numpy
            labels.append(labs.numpy())             # label numpy

    return np.concatenate(features), np.concatenate(labels)  # gộp tất cả batch


In [ ]:
train_features, train_labels = extract_features(train_loader)
test_features, test_labels = extract_features(test_loader)

print("Train features:", train_features.shape)
print("Test features:", test_features.shape)


Train features: (18639, 1000)
Test features: (4660, 1000)


In [ ]:
from sklearn import svm, metrics

clf = svm.SVC(kernel="rbf", gamma="scale", C=10)

print("Training SVM on Resnet50 features...")
clf.fit(train_features, train_labels)

print("Predicting...")
pred = clf.predict(test_features)

print(metrics.classification_report(test_labels, pred))
print(metrics.confusion_matrix(test_labels, pred))


Training SVM on InceptionResnetV1 features...
Predicting...
              precision    recall  f1-score   support

           0       0.39      0.41      0.40       806
           1       0.49      0.54      0.51       868
           2       0.35      0.36      0.36       785
           3       0.55      0.51      0.53       739
           4       0.39      0.31      0.34       653
           5       0.33      0.34      0.34       809

    accuracy                           0.42      4660
   macro avg       0.42      0.41      0.41      4660
weighted avg       0.42      0.42      0.42      4660

[[334  98 128  61  67 118]
 [ 91 471  96  80  39  91]
 [136 128 286  47  51 137]
 [ 77  82  70 377  65  68]
 [ 91  69  98  62 200 133]
 [126 116 135  60  97 275]]


In [ ]:
import numpy as np

# Đường dẫn thư mục trong Drive
save_path = "/content/drive/MyDrive/CREMA-D/"  # đổi thành thư mục bạn muốn

# Tạo thư mục nếu chưa có
import os
os.makedirs(save_path, exist_ok=True)

# Ví dụ InceptionResnetV1
np.save(save_path + "train_features_resnet50.npy", train_features)
np.save(save_path + "train_labels_resnet50.npy", train_labels)
np.save(save_path + "test_features_resnet50.npy", test_features)
np.save(save_path + "test_labels_resnet50.npy", test_labels)

print("Đã lưu train/test features InceptionResnetV1 vào Drive!")

print("Đã lưu train/test features vào Drive!")


Đã lưu train/test features InceptionResnetV1 vào Drive!
Đã lưu train/test features vào Drive!


#tương tự với resnet18 và  InceptionResnetV1 (FaceNet)  đã lưu vào drive từ trước

In [ ]:
import numpy as np

save_path = "/content/drive/MyDrive/CREMA-D/"

# =======================
# 1) ResNet18
# =======================
X_train_r18 = np.load(save_path + "train_features.npy")
y_train_r18 = np.load(save_path + "train_labels.npy")
X_test_r18 = np.load(save_path + "test_features.npy")
y_test_r18 = np.load(save_path + "test_labels.npy")

# =======================
# 2) InceptionResnetV1 (FaceNet)
# =======================
X_train_incep = np.load(save_path + "train_features_incep.npy")
y_train_incep = np.load(save_path + "train_labels_incep.npy")
X_test_incep = np.load(save_path + "test_features_incep.npy")
y_test_incep = np.load(save_path + "test_labels_incep.npy")

# =======================
# 3) ResNet50
# =======================
X_train_r50 = np.load(save_path + "train_features_resnet50.npy")
y_train_r50 = np.load(save_path + "train_labels_resnet50.npy")
X_test_r50 = np.load(save_path + "test_features_resnet50.npy")
y_test_r50 = np.load(save_path + "test_labels_resnet50.npy")


so sách 3 pretrain feature cho svm

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score

print("===== RESNET18 =====")
clf_r18 = SVC(kernel='linear').fit(X_train_r18, y_train_r18)
pred_r18 = clf_r18.predict(X_test_r18)
print("Accuracy:", accuracy_score(y_test_r18, pred_r18))
print("F1-score:", f1_score(y_test_r18, pred_r18, average='macro'))

print("\n===== INCEPTION RESNET V1 =====")
clf_incep = SVC(kernel='linear').fit(X_train_incep, y_train_incep)
pred_incep = clf_incep.predict(X_test_incep)
print("Accuracy:", accuracy_score(y_test_incep, pred_incep))
print("F1-score:", f1_score(y_test_incep, pred_incep, average='macro'))

print("\n===== RESNET50 =====")
clf_r50 = SVC(kernel='linear').fit(X_train_r50, y_train_r50)
pred_r50 = clf_r50.predict(X_test_r50)
print("Accuracy:", accuracy_score(y_test_r50, pred_r50))
print("F1-score:", f1_score(y_test_r50, pred_r50, average='macro'))


===== RESNET18 =====
Accuracy: 0.43540772532188843
F1-score: 0.432236909123378

===== INCEPTION RESNET V1 =====
Accuracy: 0.30107296137339057
F1-score: 0.292763142498587

===== RESNET50 =====
Accuracy: 0.4253218884120172
F1-score: 0.4229821029414453


In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import numpy as np

def plot_tsne_compare(Xr18, Xi, Xr50, title):
    X_all = np.concatenate([Xr18, Xi, Xr50], axis=0)
    feat_type = np.array([0]*len(Xr18) + [1]*len(Xi) + [2]*len(Xr50))

    X_2d = TSNE(n_components=2, random_state=42, perplexity=30, learning_rate=200).fit_transform(X_all)

    plt.figure(figsize=(8,8))
    plt.scatter(X_2d[feat_type==0,0], X_2d[feat_type==0,1], s=5, label='ResNet18')
    plt.scatter(X_2d[feat_type==1,0], X_2d[feat_type==1,1], s=5, label='InceptionResNetV1')
    plt.scatter(X_2d[feat_type==2,0], X_2d[feat_type==2,1], s=5, label='ResNet50')
    plt.title(title)
    plt.legend()
    plt.show()

plot_tsne_compare(X_test_r18, X_test_incep, X_test_r50, "So sánh 3 loại Feature bằng TSNE")


NameError: name 'X_test_r18' is not defined